# TRAINING

In [1]:
# 1. NVIDIA-Treiber installieren (aktuellste Version, Download von NVIDIA.com)
# 2. CUDA Toolkit installieren (https://developer.nvidia.com/cuda-downloads)
# 3. Ultralytics installieren (automatisch inkl. torch/cuda support)
!python -m pip install --upgrade pip
%pip install ultralytics --upgrade
#%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

  Using cached pip-25.1.1-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-25.1.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 24.3 MB/s eta 0:00:00
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)
Using cached ultralytics_thop-2.0.14-py3-none-any.whl (26 kB)
Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)

   ---------------------------------------- 0/4 [py-cpuinfo]
   ---------- ----------------------------- 1/4 [opencv-python]

Note: you may need to restart the kernel to use updated packa

ERROR: Could not install packages due to an OSError: [WinError 5] Zugriff verweigert: 'c:\\Users\\jonni\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



Nach der Installation prüfe in Python, ob CUDA verfügbar ist:

In [2]:
import torch
print(torch.cuda.is_available())  # True = CUDA bereit


True


# Pfad zur YAML Datei eintragen

Passe bei Bedarf die Parameter an (epochs, batch, imgsz …)

Starte das Skript – Training läuft!

In [5]:
!pip install --user ultralytics


  Using cached ultralytics-8.3.148-py3-none-any.whl.metadata (37 kB)
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
Using cached ultralytics-8.3.148-py3-none-any.whl (1.0 MB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)
Using cached ultralytics_thop-2.0.14-py3-none-any.whl (26 kB)

   ---------------------------------------- 0/3 [opencv-python]
   ---------------------------------------- 0/3 [opencv-python]
   ---------------------------------------- 0/3 [opencv-python]
   ---------------------------------------- 0/3 [opencv-python]
   ------------- -------------------------- 1/3 [ultralytics-thop]
   -------------------------- ------------- 2/3 [ultralytics]
   -------------------------- ------------- 2/3 [ultralytics]
   -------------------------- ------------- 2/3 [ultralytics]
   -------------------------- ------------- 2/3 [ultralytics]
   ----------

In [6]:
from ultralytics import YOLO

# --- Einstellungen (bitte ggf. anpassen) ---

MODEL_PATH = 'yolo11n.pt'               # Nano Modell, schnell und ressourcensparend (Standard)

DATASET_YAML = r"C:\Users\jonni\OneDrive\Dokumente\GitHub\Chairlift_Gefahrenerkennung\01_Data_Jon\Augmented\Images_split\Images\dataset.yaml"   # Pfad zur dataset.yaml (wie vorhin erzeugt)


### Trainings-Argumente einstellen

| Argument       | Bedeutung (einfach erklärt)                                                          | Default           |
| -------------- | ------------------------------------------------------------------------------------ | ----------------- |
| `epochs`       | Wie oft wird der ganze Datensatz "durchtrainiert"? (Mehr = besser, aber auch länger) | 100               |
| `imgsz`        | Bildgrösse, auf die vor Training skaliert wird.                                      | 640               |
| `batch`        | Wie viele Bilder werden gleichzeitig an die GPU/CPU geschickt?                       | 16                |
| `device`       | 0 = erste GPU, 1 = zweite GPU, 'cpu' = Prozessor                                     | 0                 |
| `optimizer`    | Optimierungsverfahren, das das Lernen steuert.                                       | 'auto'            |
| `lr0`          | Startwert der Lernrate                                                               | 0.01              |
| `lrf`          | Endwert der Lernrate, als Anteil von lr0                                             | 0.01              |
| `momentum`     | Schwung der Gewichtsaktualisierung (wichtig bei SGD)                                 | 0.937             |
| `weight_decay` | Bestraft zu komplexe Modelle (Überanpassungsschutz)                                  | 0.0005            |
| `patience`     | Wieviele Epochen abwarten, wenn keine Verbesserung mehr kommt?                       | 50                |
| `cos_lr`       | Nutze Cosine-Learningrate, oft für längeres, sanfteres Lernen                        | False             |
| `project`      | Ordner, in dem alles gespeichert wird                                                | 'runs/train'      |
| `name`         | Name des Experiments (wird im Projektordner gespeichert)                             | 'yolo11n\_custom' |
| `pretrained`   | Beginne mit vortrainiertem Modell (schneller & meist bessere Ergebnisse)             | True              |
| `resume`       | Falls Training abbricht, kannst du damit weitermachen                                | False             |
| `val`          | Nach jedem Durchgang das Modell testen/validieren                                    | True              |
| `workers`      | CPU-Prozesse fürs Datenladen (mehr ist schneller, je nach PC)                        | 4                 |


In [7]:
# Trainingsparameter (alle wichtigsten Argumente, mit Defaults)
TRAIN_ARGS = {
    'epochs': 100,           # Wie oft das ganze Dataset gesehen wird (Standard: 100)
    'imgsz': 640,            # Bildgrösse, Standard ist 640 (je nach GPU bis 1280 möglich)
    'batch': 32,             # Batch-Grösse, Anzahl Bilder pro Schritt (je nach VRAM 8–32)
    'device': 0,             # GPU-Index (0=erste GPU, 'cpu'=CPU nutzen, [0,1] für mehrere GPUs)
    'optimizer': 'auto',     # Optimizer (z.B. SGD, Adam, AdamW oder 'auto' für Automatik)
    'lr0': 0.01,             # Start-Lernrate
    'lrf': 0.01,             # End-Lernrate als Faktor von lr0. Also 0.01 = 1% von lr0
    'momentum': 0.937,       # Nur für SGD, "Schwung" der Updates
    'weight_decay': 0.0005,  # Regularisierung gegen Überanpassung
    'patience': 50,          # Stoppt Training, falls Val-Loss nicht besser wird (Epochen)
    'cos_lr': False,         # Cosine-Learningrate statt linear (True/False)
    'project': 'runs/train', # Wohin die Ergebnisse gespeichert werden
    'name': 'yolo11n_custom',# Name des Experiments (wird als Ordner erzeugt)
    'pretrained': True,      # Pretrained Modell nutzen (empfohlen)
    'resume': False,         # Training fortsetzen, falls gestoppt
    'val': True,             # Nach jedem Epoch validieren (empfohlen)
    'workers': 14,            # Anzahl CPU-Worker fürs Laden (je nach PC 2–8)
    
    # Data Augmentation direkt im Training (Stärke, Methode)
    'hsv_h': 0.015,             # Farbtonverschiebung (0.0–0.5), 0=aus
    'hsv_s': 0.7,               # Sättigung (0.0–0.9)
    'hsv_v': 0.4,               # Helligkeit (0.0–0.9)
    'degrees': 0.0,             # Rotation in Grad, z. B. 10
    'translate': 0.1,           # Verschiebung als Anteil, z. B. 0.1 = 10%
    'scale': 0.5,               # Skalierung (0=aus, 0.5=±50%)
    'shear': 0.0,               # Scherung (0–2.0)
    'perspective': 0.0,         # Perspektivische Verzerrung (0–0.001)
    'flipud': 0.0,              # Vertikal spiegeln (Wahrscheinlichkeit, 0–1)
    'fliplr': 0.5,              # Horizontal spiegeln (Wahrscheinlichkeit, 0–1)
    'mosaic': 1.0,              # Mosaic-Augmentation (0–1), meist anlassen
    'mixup': 0.0,               # Mixup (0–1), Bilder kombinieren

    # Loss-Anteile für Boxen, Klassifikation, Objekt
    'box': 7.5,                 # Box-Loss-Gewichtung
    'cls': 0.5,                 # Klassifikations-Loss-Gewichtung
    'dfl': 1.5,                 # Distribution Focal Loss (0–10)

    # Early stopping deaktivieren
    'patience': 50,            # Höhere Zahl = weniger frühes Abbrechen (Standard: 50)

    # Save/checkpoints
    'save_period': 10,          # Alle x Epochen ein Modell speichern (Default=50)
    'exist_ok': True,           # Vorhandene Ergebnisse überschreiben (False=Fehler)
    'verbose': True,            # Detailierte Ausgaben (False=weniger Text)

    # Advanced: Training nur auf bestimmten Klassen (nur falls gebraucht)
    # 'classes': [0, 2],        # Nur Klasse 0 und 2 trainieren

    # Seed für Reproduzierbarkeit
    'seed': 42                 # Zufallszahlgenerator initialisiere
}

---
### Training startet mit Ausführung des nächsten Skriptes

In [8]:
# ---- Schritt 1: Modell laden ----
model = YOLO(MODEL_PATH)

# ---- Schritt 2: Training starten ----
results = model.train(
    data=DATASET_YAML,
    epochs=TRAIN_ARGS['epochs'],
    imgsz=TRAIN_ARGS['imgsz'],
    batch=TRAIN_ARGS['batch'],
    device=TRAIN_ARGS['device'],
    optimizer=TRAIN_ARGS['optimizer'],
    lr0=TRAIN_ARGS['lr0'],
    lrf=TRAIN_ARGS['lrf'],
    momentum=TRAIN_ARGS['momentum'],
    weight_decay=TRAIN_ARGS['weight_decay'],
    patience=TRAIN_ARGS['patience'],
    cos_lr=TRAIN_ARGS['cos_lr'],
    project=TRAIN_ARGS['project'],
    name=TRAIN_ARGS['name'],
    pretrained=TRAIN_ARGS['pretrained'],
    resume=TRAIN_ARGS['resume'],
    val=TRAIN_ARGS['val'],
    workers=TRAIN_ARGS['workers'],
    hsv_h=TRAIN_ARGS['hsv_h'],
    hsv_s=TRAIN_ARGS['hsv_s'],
    hsv_v=TRAIN_ARGS['hsv_v'],
    degrees=TRAIN_ARGS['degrees'],
    translate=TRAIN_ARGS['translate'],
    scale=TRAIN_ARGS['scale'],
    shear=TRAIN_ARGS['shear'],
    perspective=TRAIN_ARGS['perspective'],
    flipud=TRAIN_ARGS['flipud'],
    fliplr=TRAIN_ARGS['fliplr'],
    mosaic=TRAIN_ARGS['mosaic'],
    mixup=TRAIN_ARGS['mixup'],
    box=TRAIN_ARGS['box'],
    cls=TRAIN_ARGS['cls'],
    dfl=TRAIN_ARGS['dfl'],
    save_period=TRAIN_ARGS['save_period'],
    exist_ok=TRAIN_ARGS['exist_ok'],
    verbose=TRAIN_ARGS['verbose'],
    # classes=TRAIN_ARGS['classes'],  # Nur wenn nötig
    seed=TRAIN_ARGS['seed']
)

# Das Training wird jetzt gestartet. Je nach Hardware kann das einige Zeit dauern.
# Die Ergebnisse werden im angegebenen Projekt-Ordner gespeichert.

Ultralytics 8.3.148  Python-3.12.10 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\jonni\OneDrive\Dokumente\GitHub\Chairlift_Gefahrenerkennung\01_Data_Jon\Augmented\Images_split\Images\dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_custom, nbs=64, nms=False, op

100%|██████████| 755k/755k [00:00<00:00, 18.2MB/s]

Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

YOLO11n summary: 181 layers, 2,590,425 parameters, 2,590,409 gradients, 6.4 GFLOPs

Transferred 448/499 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed 
train: Fast image access  (ping: 0.10.0 ms, read: 4.41.2 MB/s, size: 38.5 KB)


train: Scanning C:\Users\jonni\OneDrive\Dokumente\GitHub\Chairlift_Gefahrenerkennung\01_Data_Jon\Augmented\Images_split\Images\train\labels... 315 images, 0 backgrounds, 100 corrupt: 100%|██████████| 315/315 [00:00<00:00, 615.87it/s]

...

    100/100      4.18G     0.6541     0.4926     0.8705         36        640: 100%|██████████| 7/7 [00:05<00:00,  1.38it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.45it/s]

                   all         57         94      0.992      0.969      0.989      0.775



100 epochs completed in 0.211 hours.
Optimizer stripped from runs\train\yolo11n_custom\weights\last.pt, 5.5MB
Optimizer stripped from runs\train\yolo11n_custom\weights\best.pt, 5.5MB

Validating runs\train\yolo11n_custom\weights\best.pt...
Ultralytics 8.3.148  Python-3.12.10 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.66it/s]


                   all         57         94      0.992      0.968      0.991      0.802
                KLEBER         50         70      0.996      0.914      0.982      0.803
                EDDING         10         11       0.98          1      0.995      0.795
                TACKER         13         13          1      0.989      0.995      0.807
Speed: 0.3ms preprocess, 4.9ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs\train\yolo11n_custom
